In [3]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import os
import sys

## Configurando pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# VARIAVEIS PADRÃO

In [149]:
today = datetime.now()
extract_date = today.strftime('%Y-%m-%d')
period = today.strftime('%m')
year = today.strftime('%Y')
day = today.strftime('%d')

# extract_date = '2024-05-20'
# period = '05'
# year = '2024'
# day = '20'


print(f'Hoje: {extract_date}, period: {period}, Year: {year}, day: {day}')

Hoje: 2024-05-23, period: 05, Year: 2024, day: 23


# Tabelas do banco

## Tabela cabeçalho de conta

Tabela será carregada apenas uma vez e não terá mais seus dados alterados depois do primeiro cadastro, a não ser por pedido dos clientes

In [4]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\tbAccount_ig.json'
data_cabecalho = pd.read_json(file_path)
data_cabecalho.head()

rename_column_cabecalho = {
    'id': 'id_account',
    'username': 'account_username',
    'biography': 'account_biography',
    'profile_picture_url': 'profile_picture_url',
    'name': 'account_name'
}
data_cabecalho.rename(columns=rename_column_cabecalho, inplace=True)

select_column_cabecalho = ['id_account', 'account_username', 'account_biography', 'profile_picture_url', 'account_name', 'followers_count', 'follows_count', 'media_count']
data_cabecalho = data_cabecalho[select_column_cabecalho]
data_cabecalho.head()

,id_account,account_username,account_biography,profile_picture_url,account_name,followers_count,follows_count,media_count
0,17841425444516188,bodemeier.digital,♟️Criação de conteúdo estratégico\n▪️+ de 5 an...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Bodemeier Digital,2876,1399,90
1,17841417530400616,gabgalani,RJ 🔸 SP\ndata is the new oil 💻\n@carolinebodem...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Gabriel Galani,239,374,139


Salvando arquivo tratado

In [151]:
cabecalho_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCabecalho.csv'
data_cabecalho.to_csv(cabecalho_output, sep='\t')

## Tabela de midias

Terá uma chave única baseada na conta e data da extração

In [199]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMedias.json'
data_midias_sem_insights = pd.read_json(file_path, orient='records', dtype={'id': str})
data_midias_sem_insights
dataframes = []
owner_df = pd.json_normalize(data_midias_sem_insights['owner']).rename(columns={'id': 'id_account'})
df_final_midia = pd.concat([data_midias_sem_insights, owner_df], axis=1)
df_final_midia.drop(columns=['owner'], inplace=True)
df_final_midia.rename(columns={'id': 'id_midia'}, inplace=True)

select_tb_midia = ['id_account', 'username', 'id_midia', 'comments_count', 'like_count',
            'media_type', 'media_url', 'caption', 'timestamp', 'permalink',
            'media_product_type', 'thumbnail_url', 'shortcode']
df_tb_midia = df_final_midia[select_tb_midia]
df_tb_midia['extract_date'] = extract_date
df_tb_midia['period'] = period
df_tb_midia['year'] = year
df_tb_midia['day'] = day
df_tb_midia['id_tb_midia'] = df_tb_midia['id_midia'].astype(str) + df_tb_midia['year'] + df_tb_midia['period'] + df_tb_midia['day']

# filtro = df_tb_midia[df_tb_midia['id_midia'] == '18002752616172667']
df_tb_midia.head()

,id_account,username,username,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,shortcode,extract_date,period,year,day,id_tb_midia
0,17841425444516188,bodemeier.digital,bodemeier.digital,18005273918369940,2,15,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Você sabe o que é um briefing? 📝\n\n🎥 Assista ...,2024-02-20 21:19:22+00:00,https://www.instagram.com/reel/C3lW1XkOcfL/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3lW1XkOcfL,2024-05-23,05,2024,23,1800527391836994020240523
1,17841425444516188,bodemeier.digital,bodemeier.digital,17846370906157882,1,13,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Uma grande dúvida que vocês me perguntam: SOCI...,2024-02-16 21:00:00+00:00,https://www.instagram.com/reel/C3bDRy3uifD/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3bDRy3uifD,2024-05-23,05,2024,23,1784637090615788220240523
2,17841425444516188,bodemeier.digital,bodemeier.digital,18002752616172667,0,5,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,“O maior instrumento da globalização cultural ...,2024-02-10 20:05:55+00:00,https://www.instagram.com/reel/C3Lf9wLOhpf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3Lf9wLOhpf,2024-05-23,05,2024,23,1800275261617266720240523
3,17841425444516188,bodemeier.digital,bodemeier.digital,17971594208670851,0,7,VIDEO,NaN,‼️ É isso que o Instagram prioriza!\n\nAcredit...,2024-02-09 18:00:00+00:00,https://www.instagram.com/reel/C3KvokquZdf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3KvokquZdf,2024-05-23,05,2024,23,1797159420867085120240523
4,17841425444516188,bodemeier.digital,bodemeier.digital,18062694892504235,5,7,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,✨Aqui está: \n\nMas primeiro já me segue para ...,2024-02-09 13:00:00+00:00,https://www.instagram.com/reel/C3IK23sudbS/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3IK23sudbS,2024-05-23,05,2024,23,1806269489250423520240523


In [200]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMedias.json'
# data_midias_sem_insights = pd.read_json(file_path)
# rename = {
#     'id': 'id_account'
# }
# data_midias_sem_insights.rename(columns=rename, inplace=True)

# # df_normalized = pd.json_normalize(data_midias_sem_insights['media'])
# # df_normalized = pd.json_normalize(df_normalized['data'])
# # df = pd.concat([data_midias_sem_insights, df_normalized], axis=1)
# df.drop(columns=['id_account','media','username'], inplace=True)

# dataframes = []
# for i in df.columns:
#     df_json = pd.json_normalize(df[i])
#     df_json = pd.concat([data_midias_sem_insights, df_json], axis=1)
#     dataframes.append(df_json)

    
# for i, df in enumerate(dataframes):
#     dataframes[i] = df.loc[:, ~df.columns.duplicated()]
# df_final = pd.concat(dataframes,ignore_index=True)
# df_final.rename(columns={'id': 'id_midia'}, inplace=True)

# select_tb_midia = ['id_account', 'username', 'id_midia', 'comments_count', 'like_count',
#        'media_type', 'media_url', 'caption', 'timestamp', 'permalink',
#        'media_product_type', 'thumbnail_url', 'shortcode', 'owner.id',
#        'owner.username']
# df_tb_midia = df_final[select_tb_midia]
# df_tb_midia['extract_date'] = extract_date
# df_tb_midia['period'] = period
# df_tb_midia['year'] = year
# df_tb_midia['day'] = day
# df_tb_midia['id_tb_midia'] = df_tb_midia['id_midia'] + df_tb_midia['year'] + df_tb_midia['period'] + df_tb_midia['day']
# df_tb_midia.head()

Salvando arquivo tratado

In [201]:
midia_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMedias.csv'
df_final_midia.to_csv(midia_output, sep='\t')

#### Tb Carrocel (Utiliza o dataset que é usado na tb de midia)

Só será carregada caso tenham carroceus novos

In [202]:
df_final_midia.head()

,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,shortcode,username,children,id_account,username
0,18005273918369940,2,15,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Você sabe o que é um briefing? 📝\n\n🎥 Assista ...,2024-02-20 21:19:22+00:00,https://www.instagram.com/reel/C3lW1XkOcfL/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3lW1XkOcfL,bodemeier.digital,NaN,17841425444516188,bodemeier.digital
1,17846370906157882,1,13,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Uma grande dúvida que vocês me perguntam: SOCI...,2024-02-16 21:00:00+00:00,https://www.instagram.com/reel/C3bDRy3uifD/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3bDRy3uifD,bodemeier.digital,NaN,17841425444516188,bodemeier.digital
2,18002752616172667,0,5,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,“O maior instrumento da globalização cultural ...,2024-02-10 20:05:55+00:00,https://www.instagram.com/reel/C3Lf9wLOhpf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3Lf9wLOhpf,bodemeier.digital,NaN,17841425444516188,bodemeier.digital
3,17971594208670851,0,7,VIDEO,NaN,‼️ É isso que o Instagram prioriza!\n\nAcredit...,2024-02-09 18:00:00+00:00,https://www.instagram.com/reel/C3KvokquZdf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3KvokquZdf,bodemeier.digital,NaN,17841425444516188,bodemeier.digital
4,18062694892504235,5,7,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,✨Aqui está: \n\nMas primeiro já me segue para ...,2024-02-09 13:00:00+00:00,https://www.instagram.com/reel/C3IK23sudbS/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3IK23sudbS,bodemeier.digital,NaN,17841425444516188,bodemeier.digital


In [203]:
# Selecionar apenas as colunas desejadas e filtrar por 'media_type' igual a 'CAROUSEL_ALBUM'
select_carrocel = ['id_midia', 'media_type', 'children']
df_carroceu = df_final_midia[select_carrocel]
df_carroceu = df_carroceu[df_carroceu['media_type'] == "CAROUSEL_ALBUM"]
normalized_df = pd.json_normalize(df_carroceu['children'])
normalized_df = normalized_df.explode('data')
expanded_df = normalized_df.drop(columns='data').join(pd.json_normalize(normalized_df['data']))

dataframe_carrocel = pd.concat([df_carroceu.reset_index(drop=True), expanded_df.reset_index(drop=True)], axis=1)
dataframe_carrocel.drop(columns='children', inplace=True)
dataframe_carrocel.head()

# # Itera sobre cada linha do DataFrame
# for i, row in df_carroceu.iterrows():
#     # Normaliza a coluna 'children.data' e adiciona uma coluna com o índice original
#     normalized_df = pd.json_normalize(row['children'])
#     normalized_df = pd.json_normalize(row['data'])
#     normalized_df['index'] = i
#     normalized_dfs.append(normalized_df)

# # Concatena todos os DataFrames normalizados
# normalized_df = pd.concat(normalized_dfs)

# # Une o DataFrame normalizado ao DataFrame original
# # Isso resulta em duplicação de linhas para cada item da lista dentro do 'children.data'
# result_df = df_carroceu.drop(columns=['children']).merge(normalized_df, left_index=True, right_on='index')

# # Remova a coluna 'index' se não for mais necessária
# dataframe_carrocel = result_df.drop(columns=['index'])


,id_midia,media_type,media_type,media_url,id
0,18024592819898084,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t39.30808-...,17971829027659357
1,17950801427740207,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t39.30808-...,17971829027659357
2,18249982000212022,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t39.30808-...,17971829027659357
3,17970826016660160,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t39.30808-...,17971829027659357
4,17983677587408251,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t39.30808-...,17971829027659357


Salvando arquivo tratado

In [157]:
carrocel_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCarrocel.csv'
dataframe_carrocel.to_csv(carrocel_output, sep='\t')

## Tb Stories (criar uma chave unica, baseada na conta e data da extração)

In [158]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStories.json'
# data_stories_sem_insights = pd.read_json(file_path)
data_stories_sem_insights = pd.read_json(file_path, orient='records', dtype={'id': str})
data_stories_sem_insights.dropna(subset=['owner'], inplace=True)
owner_df = pd.json_normalize(data_stories_sem_insights['owner']).rename(columns={'id': 'id_account'})
df_final_midia = pd.concat([data_stories_sem_insights.reset_index(drop=True), owner_df.reset_index(drop=True)], axis=1)
df_final_midia.drop(columns=['owner'], inplace=True)
df_final_midia.rename(columns={'id': 'id_midia'}, inplace=True)

df_final_midia['extract_date'] = extract_date
df_final_midia['period'] = period
df_final_midia['year'] = year
df_final_midia['day'] = day
df_final_midia['id_tb_midia'] = df_final_midia['id_midia'].astype(str) + df_final_midia['year'] + df_final_midia['period'] + df_final_midia['day']


df_final_midia.head()

,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,shortcode,username,id_account,username,extract_date,period,year,day,id_tb_midia
0,18029303027098233,0.0,0.0,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,@carolinebodemeier,2024-05-23 10:14:38+00:00,https://instagram.com/stories/gabgalani/337422...,STORY,C7Tqf9UuJmY,gabgalani,17841417530400616,gabgalani,2024-05-23,05,2024,23,1802930302709823320240523


In [159]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStories.json'
# data_stories_sem_insights = pd.read_json(file_path)

# data_stories_json = pd.json_normalize(data_stories_sem_insights['stories'])
# # data_stories_json = pd.json_normalize(data_stories_json['data'])
# data_stories_json = pd.concat([data_stories_sem_insights, data_stories_json], axis=1)
# data_stories_json = data_stories_json[['id', 'data']]
# data_stories_normalizado = data_stories_json.dropna()

# normalized_dfs = []

# # Itera sobre cada linha do DataFrame
# for i, row in data_stories_normalizado.iterrows():
#     # Normaliza a coluna 'children.data' e adiciona uma coluna com o índice original
#     normalized_df = pd.json_normalize(row['data'])
#     normalized_df['index'] = i
#     normalized_dfs.append(normalized_df)

# # Concatena todos os DataFrames normalizados
# normalized_df = pd.concat(normalized_dfs)

# # Une o DataFrame normalizado ao DataFrame original
# # Isso resulta em duplicação de linhas para cada item da lista dentro do 'children.data'
# result_df = data_stories_normalizado.drop(columns=['data']).merge(normalized_df, left_index=True, right_on='index')

# # Remova a coluna 'index' se não for mais necessária
# data_stories_normalizado = result_df.drop(columns=['index'])
# data_stories_normalizado.rename(columns={'id_x': 'id_account', 'id_y': 'id_story'}, inplace=True)


# data_stories_normalizado['extract_date'] = extract_date
# data_stories_normalizado['period'] = period
# data_stories_normalizado['year'] = year
# data_stories_normalizado['day'] = day
# data_stories_normalizado['id_tb_midia'] = data_stories_normalizado['id_story'] + data_stories_normalizado['year'] + data_stories_normalizado['period'] + data_stories_normalizado['day']


# data_stories_normalizado.head()

Saldo arquivo tratado

In [160]:
stories_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStories.csv'
df_final_midia.to_csv(stories_output, sep='\t')

## Tb Account Day Insights

In [161]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbDayAccounIgMetrics.json'
data_day_account = pd.read_json(file_path)

columns = pd.json_normalize(data_day_account['values'])
column_0 = pd.json_normalize(columns[0])
column_0.rename(columns={'value': 'last_day', 'end_time': 'last_end_time'}, inplace=True)
column_1 = pd.json_normalize(columns[1])
column_1.rename(columns={'value': 'actual_day', 'end_time': 'actual_end_time'}, inplace=True)
resultado = pd.concat([data_day_account, column_0, column_1], axis=1)

resultado['id_account'] = resultado['id'].str.split('/').str[0]
resultado['chave'] = resultado['id_account'].astype(str) + resultado['name'] 

resultado['extract_date'] = extract_date
resultado['period_extraction'] = period
resultado['year'] = year
resultado['day'] = day
resultado['id_tb_midia'] = (
    resultado['id_account'].astype(str) + 
    resultado['year'].astype(str) + 
    resultado['period_extraction'].astype(str) + 
    resultado['day'].astype(str) + 
    resultado['name'].astype(str) + 
    resultado['period'].astype(str)
)

resultado.drop(columns='values',inplace=True)

resultado.head()

,name,period,title,description,id,username,last_day,last_end_time,actual_day,actual_end_time,id_account,chave,extract_date,period_extraction,year,day,id_tb_midia
0,impressions,day,Impressões,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/impressions/day,bodemeier.digital,0,2024-05-22T07:00:00+0000,2,2024-05-23T07:00:00+0000,17841425444516188,17841425444516188impressions,2024-05-23,05,2024,23,1784142544451618820240523impressionsday
1,reach,day,Alcance,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/reach/day,bodemeier.digital,0,2024-05-22T07:00:00+0000,1,2024-05-23T07:00:00+0000,17841425444516188,17841425444516188reach,2024-05-23,05,2024,23,1784142544451618820240523reachday
2,follower_count,day,Número de seguidores,Número total de contas únicas que seguem este ...,17841425444516188/insights/follower_count/day,bodemeier.digital,0,2024-05-22T07:00:00+0000,0,2024-05-23T07:00:00+0000,17841425444516188,17841425444516188follower_count,2024-05-23,05,2024,23,1784142544451618820240523follower_countday
3,email_contacts,day,Contatos de email,Número total de toques no link de email deste ...,17841425444516188/insights/email_contacts/day,bodemeier.digital,0,2024-05-22T07:00:00+0000,0,2024-05-23T07:00:00+0000,17841425444516188,17841425444516188email_contacts,2024-05-23,05,2024,23,1784142544451618820240523email_contactsday
4,phone_call_clicks,day,Cliques em ligação telefônica,Número total de toques no link de ligação dest...,17841425444516188/insights/phone_call_clicks/day,bodemeier.digital,0,2024-05-22T07:00:00+0000,0,2024-05-23T07:00:00+0000,17841425444516188,17841425444516188phone_call_clicks,2024-05-23,05,2024,23,1784142544451618820240523phone_call_clicksday


In [162]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbDayAccounIgMetrics.json'
# data_day_account = pd.read_json(file_path)

# df_json = pd.json_normalize(data_day_account['insights'])
# df_json = pd.concat([data_day_account, df_json], axis=1)
# df_json.rename (columns={'id': 'id_account'}, inplace=True)
# df_json = df_json[['id_account','data']]

# df_primeira = pd.json_normalize(df_json['data'])

# dfs = []
# for c in df_primeira.columns: 
#     json = pd.json_normalize(df_primeira[c])
#     json_concat = pd.concat([df_json, json], axis=1)
#     dfs.append(json_concat)

# json_1_final = pd.concat(dfs)
# json_1_final['chave'] = json_1_final['id_account'].astype(str) + json_1_final['name']


# json_final = json_1_final[['id_account', 'name', 'values']]
# df_expanded = pd.concat([json_final.drop(columns='values').reset_index(drop=True), pd.json_normalize(json_final['values'])], axis=1)

# json = df_expanded[[0,1]]
# dfs = []
# for i in json.columns: 
#     columns = pd.json_normalize(json[i])
#     if i == 0:
#         columns.rename(columns={'value': 'last_day', 'end_time': 'last_end_time'}, inplace=True)
#     if i == 1:
#         columns.rename(columns={'value': 'actual_day', 'end_time': 'actual_end_time'}, inplace=True)
#     col_concat = pd.concat([df_expanded, columns], axis=1)
#     dfs.append(col_concat)

# teste_final = pd.concat(dfs, axis=1)

# select = ['id_account', 'name', 'last_day', 'last_end_time', 'actual_day', 'actual_end_time']
# df_ = teste_final[select]
# df_sem_duplicatas = df_.loc[:,~df_.columns.duplicated(keep='first')]
# df_sem_duplicatas['chave'] = df_sem_duplicatas['id_account'].astype(str) + df_sem_duplicatas['name'] 

# resultado = json_1_final.merge(df_sem_duplicatas, left_on='chave', right_on='chave')
# select = ['id_account_x', 'name_x', 'period', 'title', 'id', 'chave', 'last_day', 'last_end_time', 'actual_day', 'actual_end_time']
# resultado = resultado[select]


# # Se for pivotar tirar esse bloco
# resultado['extract_date'] = extract_date
# resultado['period_extraction'] = period
# resultado['year'] = year
# resultado['day'] = day
# resultado['id_tb_midia'] = (
#     resultado['id_account_x'].astype(str) + 
#     resultado['year'].astype(str) + 
#     resultado['period_extraction'].astype(str) + 
#     resultado['day'].astype(str) + 
#     resultado['name_x'].astype(str) + 
#     resultado['period'].astype(str)
# )


# # ## Converter colunas e linhas??????????(pivotar)
# # pivot_df = resultado.pivot_table(index=['id_account_x', 'period'], 
# #                           columns='name_x', 
# #                           values=['last_day', 'last_end_time', 'actual_day', 'actual_end_time'],
# #                           aggfunc='first')

# # # Flatten o MultiIndex das colunas
# # pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# # # Resetando o índice
# # pivot_df.reset_index(inplace=True)
# # select = ['id_account_x', 'period', 
# #  'actual_day_email_contacts', 'last_day_email_contacts', 
# #  'actual_day_follower_count', 'last_day_follower_count',
# #  'actual_day_get_directions_clicks', 'last_day_get_directions_clicks',
# #  'actual_day_impressions',  'last_day_impressions',
# # 'actual_day_phone_call_clicks', 'last_day_phone_call_clicks',
# # 'actual_day_profile_views', 'last_day_profile_views',
# # 'actual_day_reach', 'last_day_reach',
# # 'actual_day_text_message_clicks',  'last_day_text_message_clicks',
# # 'actual_day_website_clicks',   'last_day_website_clicks',
# # 'actual_end_time_email_contacts','last_end_time_email_contacts',
# # ]
# # pivot_df = pivot_df[select]
# # pivot_df.head()


# resultado.head()

In [163]:
account_day_insights_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccontDayInsights.csv'
resultado.to_csv(account_day_insights_output, sep='\t')

## Tb Account Insights Lifetime

Será serparada em algumas tabelas (Tb faixa etaria e genero)(tb localizacao pais) (tb Paises dos seguidores) (tb Cidade dos Seguidores)

In [164]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbLifetimeAccounIgMetrics.json'
data_lifetime_account = pd.read_json(file_path)
data_lifetime_account['id_account'] = data_lifetime_account['id'].str.split('/').str[0]

data_lifetime_account['extract_date'] = extract_date
data_lifetime_account['period_extraction'] = period
data_lifetime_account['year'] = year
data_lifetime_account['day'] = day
data_lifetime_account['id_tb_account'] = (
    data_lifetime_account['id_account'].astype(str) + 
    data_lifetime_account['year'].astype(str) + 
    data_lifetime_account['period_extraction'].astype(str) + 
    data_lifetime_account['day'].astype(str) + 
    data_lifetime_account['name'].astype(str) + 
    data_lifetime_account['period'].astype(str)
)

data_lifetime_account.head(10)

,name,period,values,title,description,id,username,id_account,extract_date,period_extraction,year,day,id_tb_account
0,audience_gender_age,lifetime,"[{'value': {'F.13-17': 3, 'F.18-24': 143, 'F.2...",Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,17841425444516188/insights/audience_gender_age...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_gender_ageli...
1,audience_locale,lifetime,"[{'value': {'el_GR': 1, 'it_IT': 5, 'ru_RU': 5...",Localização,Localidades por códigos de país dos seguidores...,17841425444516188/insights/audience_locale/lif...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_localelifetime
2,audience_country,lifetime,"[{'value': {'DE': 1, 'BD': 1, 'RU': 10, 'PT': ...",País do público,Países dos seguidores deste perfil,17841425444516188/insights/audience_country/li...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_countrylifetime
3,audience_city,lifetime,"[{'value': {'São Paulo, São Paulo (state)': 18...",Cidade do público,Cidades dos seguidores deste perfil,17841425444516188/insights/audience_city/lifetime,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_citylifetime
4,online_followers,lifetime,"[{'value': {}, 'end_time': '2024-05-23T07:00:0...",Seguidores online,Número total de seguidores deste perfil que es...,17841425444516188/insights/online_followers/li...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523online_followerslifetime
5,audience_gender_age,lifetime,"[{'value': {'F.18-24': 13, 'F.25-34': 41, 'F.3...",Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,17841417530400616/insights/audience_gender_age...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_gender_ageli...
6,audience_locale,lifetime,"[{'value': {'es_LA': 1, 'en_US': 10, 'pt_BR': ...",Localização,Localidades por códigos de país dos seguidores...,17841417530400616/insights/audience_locale/lif...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_localelifetime
7,audience_country,lifetime,"[{'value': {'BR': 210, 'HK': 1, 'MY': 1, 'EC':...",País do público,Países dos seguidores deste perfil,17841417530400616/insights/audience_country/li...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_countrylifetime
8,audience_city,lifetime,"[{'value': {'Planaltina, Goiás': 1, 'Ribeirão ...",Cidade do público,Cidades dos seguidores deste perfil,17841417530400616/insights/audience_city/lifetime,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_citylifetime
9,online_followers,lifetime,"[{'value': {}, 'end_time': '2024-05-23T07:00:0...",Seguidores online,Número total de seguidores deste perfil que es...,17841417530400616/insights/online_followers/li...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523online_followerslifetime


In [165]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbLifetimeAccounIgMetrics.json'
# data_lifetime_account = pd.read_json(file_path)
# data_lifetime_account.rename(columns={'id': 'id_account'}, inplace=True)

# json_data = pd.concat([data_lifetime_account, pd.json_normalize(pd.json_normalize(data_lifetime_account['insights'])['data'])], axis=1)
# json_data.drop(columns='insights', inplace=True)

# dataframes = []
# for i in json_data.columns: 
#     json_1 = pd.json_normalize(json_data[i])
#     json_2 = pd.concat([json_data, json_1], axis=1)
#     json_2.rename(columns={'values': f'values.{i}', 'title': f'title.{i}', 'description': f'description.{i}', 'name': f'name.{i}'}, inplace=True)
#     dataframes.append(json_2)

# df = pd.concat(dataframes, axis=1)
# df_sem_duplicatas = df.loc[:, ~df.columns.duplicated()]
# df_sem_duplicatas.drop(columns=[0,1,2,3,4, 'id'], inplace=True)

#### Tb faixa etaria e genero

In [166]:
df_faixa_etaria_e_genero = data_lifetime_account[data_lifetime_account['name'] == 'audience_gender_age']
values_0 = pd.json_normalize(df_faixa_etaria_e_genero['values'])
values_0 = pd.json_normalize(values_0[0])

merged_df = pd.concat([df_faixa_etaria_e_genero.reset_index(drop=True), values_0.reset_index(drop=True)], axis=1)
merged_df.drop(columns = ['values', 'id'],inplace=True)
melted_df = merged_df.melt(id_vars=['name', 'period', 'title', 'description', 'username', 'id_account', 'extract_date', 'period_extraction', 'year', 'day', 'id_tb_account'], var_name='age_gender', value_name='value')

melted_df.head()

,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_gender_ageli...,value.F.13-17,3.0
1,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_gender_ageli...,value.F.13-17,NaN
2,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_gender_ageli...,value.F.18-24,143.0
3,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_gender_ageli...,value.F.18-24,13.0
4,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_gender_ageli...,value.F.25-34,308.0


In [167]:
# value_0_select = ['id_account', 'name.0', 'period', 'values.0', 'title.0','description.0']
# value_0 = df_sem_duplicatas[value_0_select]
# normalized_values_0 = pd.json_normalize(pd.json_normalize(value_0['values.0'])[0])
# normalized_values_0 = pd.concat([value_0, normalized_values_0], axis=1)
# normalized_values_0.drop(columns='values.0',inplace=True)
# df_melted = pd.melt(normalized_values_0, 
#                     id_vars=['id_account', 'name.0', 'period', 'title.0', 'description.0'], 
#                     var_name='faixa_etaria_genero', 
#                     value_name='valor')

# df_melted['extract_date'] = extract_date
# df_melted['period_extraction'] = period
# df_melted['year'] = year
# df_melted['day'] = day
# df_melted['id_tb_midia'] = (
#     df_melted['id_account'].astype(str) + 
#     df_melted['year'].astype(str) + 
#     df_melted['period_extraction'].astype(str) + 
#     df_melted['day'].astype(str) + 
#     df_melted['name.0'].astype(str) + 
#     df_melted['period'].astype(str)
# )
# df_melted.head()

In [168]:
account_lifetime_faixa_genero = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeFaixaGenero.csv'
melted_df.to_csv(account_lifetime_faixa_genero, sep='\t')

##### Tb Localizacao Pais

In [169]:
df_localizacao_pais = data_lifetime_account[data_lifetime_account['name'] == 'audience_locale']
values_0 = pd.json_normalize(df_localizacao_pais['values'])
values_0 = pd.json_normalize(values_0[0])

merged_df = pd.concat([df_localizacao_pais.reset_index(drop=True), values_0.reset_index(drop=True)], axis=1)
merged_df.drop(columns = ['values', 'id'],inplace=True)
melted_df_1 = merged_df.melt(id_vars=['name', 'period', 'title', 'description', 'username', 'id_account', 'extract_date', 'period_extraction', 'year', 'day', 'id_tb_account'], var_name='age_gender', value_name='value')

melted_df_1.head()

,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_localelifetime,value.el_GR,1.0
1,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_localelifetime,value.el_GR,NaN
2,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_localelifetime,value.it_IT,5.0
3,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_localelifetime,value.it_IT,NaN
4,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_localelifetime,value.ru_RU,5.0


In [170]:
# value_1_select = ['id_account', 'name.1', 'period', 'values.1', 'title.1','description.1']
# value_1 = df_sem_duplicatas[value_1_select]
# normalized_values_1 = pd.json_normalize(pd.json_normalize(value_1['values.1'])[0])
# normalized_values_1 = pd.concat([value_1, normalized_values_1], axis=1)
# normalized_values_1.drop(columns='values.1',inplace=True)

# df_melted_1 = pd.melt(normalized_values_1, 
#                     id_vars=['id_account', 'name.1', 'period', 'title.1', 'description.1'], 
#                     var_name='localizacao_pais', 
#                     value_name='valor')

# df_melted_1['extract_date'] = extract_date
# df_melted_1['period_extraction'] = period
# df_melted_1['year'] = year
# df_melted_1['day'] = day
# df_melted_1['id_tb_midia'] = (
#     df_melted_1['id_account'].astype(str) + 
#     df_melted_1['year'].astype(str) + 
#     df_melted_1['period_extraction'].astype(str) + 
#     df_melted_1['day'].astype(str) + 
#     df_melted_1['name.1'].astype(str) + 
#     df_melted_1['period'].astype(str)
# )
# df_melted_1.head()

In [171]:
account_lifetime_pais = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeLocaPais.csv'
melted_df_1.to_csv(account_lifetime_pais, sep='\t')

##### tb Pais do publico

In [172]:
df_pais_do_publico = data_lifetime_account[data_lifetime_account['name'] == 'audience_country']
values_0 = pd.json_normalize(df_pais_do_publico['values'])
values_0 = pd.json_normalize(values_0[0])

merged_df = pd.concat([df_pais_do_publico.reset_index(drop=True), values_0.reset_index(drop=True)], axis=1)
merged_df.drop(columns = ['values', 'id'],inplace=True)
df_melted_2 = merged_df.melt(id_vars=['name', 'period', 'title', 'description', 'username', 'id_account', 'extract_date', 'period_extraction', 'year', 'day', 'id_tb_account'], var_name='age_gender', value_name='value')

df_melted_2.head()

,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,audience_country,lifetime,País do público,Países dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_countrylifetime,value.DE,1.0
1,audience_country,lifetime,País do público,Países dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_countrylifetime,value.DE,NaN
2,audience_country,lifetime,País do público,Países dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_countrylifetime,value.BD,1.0
3,audience_country,lifetime,País do público,Países dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_countrylifetime,value.BD,NaN
4,audience_country,lifetime,País do público,Países dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_countrylifetime,value.RU,10.0


In [173]:
# value_2_select = ['id_account', 'name.2', 'period', 'values.2', 'title.2','description.2']
# value_2 = df_sem_duplicatas[value_2_select]
# normalized_values_2 = pd.json_normalize(pd.json_normalize(value_2['values.2'])[0])
# normalized_values_2 = pd.concat([value_2, normalized_values_2], axis=1)
# normalized_values_2.drop(columns='values.2',inplace=True)
# df_melted_2 = pd.melt(normalized_values_2, 
#                     id_vars=['id_account', 'name.2', 'period', 'title.2', 'description.2'], 
#                     var_name='Pais_do_publico', 
#                     value_name='valor')

# df_melted_2['extract_date'] = extract_date
# df_melted_2['period_extraction'] = period
# df_melted_2['year'] = year
# df_melted_2['day'] = day
# df_melted_2['id_tb_midia'] = (
#     df_melted_2['id_account'].astype(str) + 
#     df_melted_2['year'].astype(str) + 
#     df_melted_2['period_extraction'].astype(str) + 
#     df_melted_2['day'].astype(str) + 
#     df_melted_2['name.2'].astype(str) + 
#     df_melted_2['period'].astype(str)
# )
# df_melted_2.head()

In [174]:
account_lifetime_pais_do_pulico = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifePaisDosSeguidores.csv'
df_melted_2.to_csv(account_lifetime_pais_do_pulico, sep='\t')

##### Tb Cidade dos Seguidores

In [175]:
df_cidade_seguidores = data_lifetime_account[data_lifetime_account['name'] == 'audience_city']
values_0 = pd.json_normalize(df_cidade_seguidores['values'])
values_0 = pd.json_normalize(values_0[0])

merged_df = pd.concat([df_cidade_seguidores.reset_index(drop=True), values_0.reset_index(drop=True)], axis=1)
merged_df.drop(columns = ['values', 'id'],inplace=True)
df_melted_3 = merged_df.melt(id_vars=['name', 'period', 'title', 'description', 'username', 'id_account', 'extract_date', 'period_extraction', 'year', 'day', 'id_tb_account'], var_name='age_gender', value_name='value')

df_melted_3.head()

,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_citylifetime,"value.São Paulo, São Paulo (state)",180.0
1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_citylifetime,"value.São Paulo, São Paulo (state)",109.0
2,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_citylifetime,"value.Karabük, Karabük Province",8.0
3,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523audience_citylifetime,"value.Karabük, Karabük Province",NaN
4,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523audience_citylifetime,"value.Rio de Janeiro, Rio de Janeiro (state)",25.0


In [176]:
# value_3_select = ['id_account', 'name.3', 'period', 'values.3', 'title.3','description.3']
# value_3 = df_sem_duplicatas[value_3_select]
# normalized_values_3 = pd.json_normalize(pd.json_normalize(value_3['values.3'])[0])
# normalized_values_3 = pd.concat([value_3, normalized_values_3], axis=1)
# normalized_values_3.drop(columns='values.3',inplace=True)
# df_melted_3 = pd.melt(normalized_values_3, 
#                     id_vars=['id_account', 'name.3', 'period', 'title.3', 'description.3'], 
#                     var_name='Cidade_dos_seguidores', 
#                     value_name='valor')

# df_melted_3['extract_date'] = extract_date
# df_melted_3['period_extraction'] = period
# df_melted_3['year'] = year
# df_melted_3['day'] = day
# df_melted_3['id_tb_midia'] = (
#     df_melted_3['id_account'].astype(str) + 
#     df_melted_3['year'].astype(str) + 
#     df_melted_3['period_extraction'].astype(str) + 
#     df_melted_3['day'].astype(str) + 
#     df_melted_3['name.3'].astype(str) + 
#     df_melted_3['period'].astype(str)
# )

# df_melted_3.head()


In [177]:
account_lifetime_cidade_do_pulico = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeCidadeDosSeguidores.csv'
df_melted_3.to_csv(account_lifetime_cidade_do_pulico, sep='\t')

### Tb Seguidores online

In [178]:
online_followers = data_lifetime_account[data_lifetime_account['name'] == 'online_followers']
values_0 = pd.json_normalize(online_followers['values'])
values_0 = pd.json_normalize(values_0[0])

merged_df = pd.concat([online_followers.reset_index(drop=True), values_0.reset_index(drop=True)], axis=1)
merged_df.drop(columns = ['values', 'id'],inplace=True)
df_melted_4 = merged_df.melt(id_vars=['name', 'period', 'title', 'description', 'username', 'id_account', 'extract_date', 'period_extraction', 'year', 'day', 'id_tb_account'], var_name='age_gender', value_name='value')

df_melted_4.head()

,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,online_followers,lifetime,Seguidores online,Número total de seguidores deste perfil que es...,bodemeier.digital,17841425444516188,2024-05-23,05,2024,23,1784142544451618820240523online_followerslifetime,end_time,2024-05-23T07:00:00+0000
1,online_followers,lifetime,Seguidores online,Número total de seguidores deste perfil que es...,gabgalani,17841417530400616,2024-05-23,05,2024,23,1784141753040061620240523online_followerslifetime,end_time,2024-05-23T07:00:00+0000


In [179]:
# value_4_select = ['id_account', 'name.4', 'period', 'values.4', 'title.4','description.4']
# value_4 = df_sem_duplicatas[value_4_select]
# normalized_values_4 = pd.json_normalize(pd.json_normalize(value_4['values.4'])[0])
# normalized_values_4 = pd.concat([value_4, normalized_values_4], axis=1)
# normalized_values_4.drop(columns='values.4',inplace=True)
# df_melted_4 = pd.melt(normalized_values_4, 
#                     id_vars=['id_account', 'name.4', 'period', 'title.4', 'description.4'], 
#                     var_name='Seguidores_online', 
#                     value_name='valor')

# df_melted_4['extract_date'] = extract_date
# df_melted_4['period_extraction'] = period
# df_melted_4['year'] = year
# df_melted_4['day'] = day
# df_melted_4['id_tb_midia'] = (
#     df_melted_4['id_account'].astype(str) + 
#     df_melted_4['year'].astype(str) + 
#     df_melted_4['period_extraction'].astype(str) + 
#     df_melted_4['day'].astype(str) + 
#     df_melted_4['name.4'].astype(str) + 
#     df_melted_4['period'].astype(str)
# )


# df_melted_4.head()

In [180]:
account_lifetime_segidores_on = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeSeguidoresOnline.csv'
df_melted_4.to_csv(account_lifetime_segidores_on, sep='\t')

## Tb Stories Insights

In [181]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStoriesInsightsLifetime.json'
data_lifetime_story = pd.read_json(file_path, orient='records', dtype={'id': str})
data_lifetime_story.dropna(subset=['insights'], inplace=True)
insights = pd.json_normalize(data_lifetime_story['insights'])
data = pd.concat([data_lifetime_story.reset_index(drop=True), insights], axis=1).drop(columns='insights')
data = pd.json_normalize(insights['data'])
data.head()

dfs = []

for index, row in data.iterrows():
    try:
        column_0 = pd.json_normalize(row)
        values = pd.json_normalize(pd.json_normalize(column_0['values'])[0])
        column_0 = pd.concat([column_0, values], axis=1).drop(columns='values')
        # column_0.rename(columns={'id': 'id_midia_tratado'}, inplace = True)
        dfs.append(column_0)
    except:
        pass
        # print(f'Passei aqui na linha {index}')
        
df_final = pd.concat(dfs).drop_duplicates().dropna()
df_final['id_midia'] = df_final['id'].str.split('/').str[0]
df_final = df_final[['id_midia', 'name', 'period', 'title','description', 'id', 'value']]

df_final['extract_date'] = extract_date
df_final['period_extraction'] = period
df_final['year'] = year
df_final['day'] = day

df_final['id_tb_midias'] = (
    df_final['id_midia'].astype(str) + 
    df_final['year'].astype(str) + 
    df_final['period_extraction'].astype(str) + 
    df_final['day'].astype(str) + 
    df_final['name'].astype(str) + 
    df_final['period'].astype(str)
)

df_final.head(10)

,id_midia,name,period,title,description,id,value,extract_date,period_extraction,year,day,id_tb_midias
0,18029303027098233,exits,lifetime,Saídas,Número de vezes que alguém saiu do story,18029303027098233/insights/exits/lifetime,12,2024-05-23,05,2024,23,1802930302709823320240523exitslifetime
1,18029303027098233,replies,lifetime,Respostas,Número total de respostas ao story,18029303027098233/insights/replies/lifetime,1,2024-05-23,05,2024,23,1802930302709823320240523replieslifetime
2,18029303027098233,taps_forward,lifetime,Toques para avançar,Número total de toques para ver a foto ou víde...,18029303027098233/insights/taps_forward/lifetime,36,2024-05-23,05,2024,23,1802930302709823320240523taps_forwardlifetime
3,18029303027098233,taps_back,lifetime,Toques para voltar,Número total de toques para ver a foto ou o ví...,18029303027098233/insights/taps_back/lifetime,1,2024-05-23,05,2024,23,1802930302709823320240523taps_backlifetime
4,18029303027098233,impressions,lifetime,Impressões,O número de vezes que seu story foi exibido na...,18029303027098233/insights/impressions/lifetime,124,2024-05-23,05,2024,23,1802930302709823320240523impressionslifetime
5,18029303027098233,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este story...,18029303027098233/insights/reach/lifetime,106,2024-05-23,05,2024,23,1802930302709823320240523reachlifetime


In [182]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMediaInsightsLifetime.json'
# data_lifetime_midia = pd.read_json(file_path)
# insights = pd.json_normalize(data_lifetime_midia['insights'])
# insights = pd.concat([data_lifetime_midia, insights], axis=1).drop(columns='insights')
# data = pd.json_normalize(insights['data'])

# dfs = []

# for index, row in data.iterrows():
#     try:
#         column_0 = pd.json_normalize(row)
#         values = pd.json_normalize(pd.json_normalize(column_0['values'])[0])
#         column_0 = pd.concat([column_0, values], axis=1).drop(columns='values')
#         datas = pd.concat([insights, column_0], axis=1).drop(columns='data')
#         dfs.append(datas)
#     except:
#         pass
#         # print(f'Passei aqui na linha {index}')
        
# df_final = pd.concat(dfs).drop_duplicates().dropna()

# df_final['extract_date'] = extract_date
# df_final['period_extraction'] = period
# df_final['year'] = year
# df_final['day'] = day

# df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# df_final['id_tb_midias'] = (
#     df_final['id'].astype(str) + 
#     df_final['year'].astype(str) + 
#     df_final['period_extraction'].astype(str) + 
#     df_final['day'].astype(str) + 
#     df_final['name'].astype(str) + 
#     df_final['period'].astype(str)
# )

# df_final.head(10)

In [183]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbStoriesInsightsLifetime.json'
# data_lifetime_story = pd.read_json(file_path)
# data_lifetime_story.dropna(subset=['stories'], inplace=True)
# data_lifetime_story.rename(columns={'id': 'id_account'}, inplace=True)

# stories_json = pd.concat([data_lifetime_story.reset_index(drop=True), pd.json_normalize(pd.json_normalize(data_lifetime_story['stories'])['data']).reset_index(drop=True)], axis=1)
# stories_json.drop(columns='stories', inplace=True)

# dfs_data = []
# for i in stories_json.columns:
#     data_json = pd.json_normalize(stories_json[i])
#     dfs_data.append(data_json)

# df_final_data = pd.concat(dfs_data)
# df_final_data.dropna(inplace=True)

# df_exploded = df_final_data.explode('insights.data').reset_index(drop=True)
# df_normalized = pd.json_normalize(df_exploded['insights.data'])
# df_final = pd.concat([df_exploded[['id']].reset_index(drop=True), df_normalized.reset_index(drop=True)], axis=1)
# df_values_normalized = pd.json_normalize(df_final['values'].explode())
# df_final = pd.concat([df_final.drop(columns=['values']), df_values_normalized], axis=1)


# df_final['extract_date'] = extract_date
# df_final['period_extraction'] = period
# df_final['year'] = year
# df_final['day'] = day

# df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# df_final['id_tb_stories'] = (
#     df_final['id'].astype(str) + 
#     df_final['year'].astype(str) + 
#     df_final['period_extraction'].astype(str) + 
#     df_final['day'].astype(str) + 
#     df_final['name'].astype(str) + 
#     df_final['period'].astype(str)
# )

# df_final.head()

In [184]:
stories_insights_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStoriesInsights.csv'
df_final.to_csv(stories_insights_output, sep='\t')

## tb Midias Insights Lifetime

In [185]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMediaInsightsLifetime.json'
data_lifetime_midia = pd.read_json(file_path)
insights = pd.json_normalize(data_lifetime_midia['insights'])
insights = pd.concat([data_lifetime_midia, insights], axis=1).drop(columns='insights')
data = pd.json_normalize(insights['data'])
data = pd.json_normalize(data[0])
data.head(10)

,name,period,values,title,description,id
0,reach,lifetime,[{'value': 146}],Contas alcançadas,O número de contas únicas que viram este reel ...,18005273918369940/insights/reach/lifetime
1,reach,lifetime,[{'value': 99}],Contas alcançadas,O número de contas únicas que viram este reel ...,17846370906157882/insights/reach/lifetime
2,reach,lifetime,[{'value': 123}],Contas alcançadas,O número de contas únicas que viram este reel ...,18002752616172667/insights/reach/lifetime
3,reach,lifetime,[{'value': 138}],Contas alcançadas,O número de contas únicas que viram este reel ...,17971594208670851/insights/reach/lifetime
4,reach,lifetime,[{'value': 138}],Contas alcançadas,O número de contas únicas que viram este reel ...,18062694892504235/insights/reach/lifetime
5,reach,lifetime,[{'value': 131}],Contas alcançadas,O número de contas únicas que viram este reel ...,17870243565059503/insights/reach/lifetime
6,reach,lifetime,[{'value': 308}],Contas alcançadas,O número de contas únicas que viram este reel ...,17897388878957061/insights/reach/lifetime
7,taps_forward,lifetime,[{'value': 0}],Toques para avançar,Número total de toques para ver a foto ou víde...,18214911715272967/insights/taps_forward/lifetime
8,reach,lifetime,[{'value': 147}],Contas alcançadas,O número de contas únicas que viram este reel ...,17905338809918494/insights/reach/lifetime
9,taps_forward,lifetime,[{'value': 0}],Toques para avançar,Número total de toques para ver a foto ou víde...,17912491130893668/insights/taps_forward/lifetime


In [194]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMediaInsightsLifetime.json'
data_lifetime_midia = pd.read_json(file_path)
insights = pd.json_normalize(data_lifetime_midia['insights'])
insights = pd.concat([data_lifetime_midia, insights], axis=1).drop(columns='insights')
data = pd.json_normalize(insights['data'])
dfs = []


dfs = []

for index, row in data.iterrows():
    try:
        column_0 = pd.json_normalize(row)
        values = pd.json_normalize(pd.json_normalize(column_0['values'])[0])
        column_0 = pd.concat([column_0, values], axis=1).drop(columns='values')
        # column_0.rename(columns={'id': 'id_midia_tratado'}, inplace = True)
        dfs.append(column_0)
    except:
        pass
        # print(f'Passei aqui na linha {index}')
        
df_final = pd.concat(dfs).drop_duplicates().dropna()
df_final.head(50)

df_final['id_midia'] = df_final['id'].str.split('/').str[0]
df_final = df_final[['id_midia', 'name', 'period', 'title','description', 'id', 'value']]

df_final['extract_date'] = extract_date
df_final['period_extraction'] = period
df_final['year'] = year
df_final['day'] = day

df_final['id_tb_midias'] = (
    df_final['id_midia'].astype(str) + 
    df_final['year'].astype(str) + 
    df_final['period_extraction'].astype(str) + 
    df_final['day'].astype(str) + 
    df_final['name'].astype(str) + 
    df_final['period'].astype(str)
)

# filtro = df_final[df_final['id_midia'] == '18002752616172667']
df_final.head()

,id_midia,name,period,title,description,id,value,extract_date,period_extraction,year,day,id_tb_midias
0,18002752616172667,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18002752616172667/insights/reach/lifetime,123.0,2024-05-23,05,2024,23,1800275261617266720240523reachlifetime
1,18002752616172667,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,18002752616172667/insights/saved/lifetime,0.0,2024-05-23,05,2024,23,1800275261617266720240523savedlifetime


In [187]:
# file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\bronze\TbMediaInsightsLifetime.json'
# data_lifetime_midia = pd.read_json(file_path)
# data_lifetime_midia.dropna(subset=['media'], inplace=True)
# data_lifetime_midia.rename(columns={'id': 'id_account'}, inplace=True)

# stories_json = pd.concat([data_lifetime_midia.reset_index(drop=True), pd.json_normalize(pd.json_normalize(data_lifetime_midia['media'])['data']).reset_index(drop=True)], axis=1)
# stories_json.drop(columns='media', inplace=True)

# dfs_data = []
# for i in stories_json.columns:
#     data_json = pd.json_normalize(stories_json[i])
#     dfs_data.append(data_json)

# df_final_data = pd.concat(dfs_data)
# df_final_data.dropna(inplace=True)


# df_exploded = df_final_data.explode('insights.data').reset_index(drop=True)
# df_normalized = pd.json_normalize(df_exploded['insights.data'])
# df_final = pd.concat([df_exploded[['id']].reset_index(drop=True), df_normalized.reset_index(drop=True)], axis=1)
# df_values_normalized = pd.json_normalize(df_final['values'].explode())
# df_final = pd.concat([df_final.drop(columns=['values']), df_values_normalized], axis=1)

# df_final['extract_date'] = extract_date
# df_final['period_extraction'] = period
# df_final['year'] = year
# df_final['day'] = day

# df_final = df_final.loc[:, ~df_final.columns.duplicated()]

# df_final['id_tb_midias'] = (
#     df_final['id'].astype(str) + 
#     df_final['year'].astype(str) + 
#     df_final['period_extraction'].astype(str) + 
#     df_final['day'].astype(str) + 
#     df_final['name'].astype(str) + 
#     df_final['period'].astype(str)
# )

# df_final.head()

In [188]:
midias_insights_output = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMidiasInsights.csv'
df_final.to_csv(midias_insights_output, sep='\t')